In [ ]:
!pip install pandas numpy scikit-learn tensorflow nltk matplotlib
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4') # Added for WordNetLemmatizer

<a href="https://colab.research.google.com/github/Chika1472/CactusFlower/blob/main/CactusFlower_Recommendation_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **추천 시스템 테스트**

### **테스트 개요**

본 시험은 텍스트 데이터를 활용해 머신러닝 및 딥러닝 모델을 설계하고 성능을 분석하는 것을 목표로 합니다. 이를 통해 텍스트 데이터의 특성을 이해하고, 데이터 탐색, 전처리, 모델 구현 및 성능 평가를 포함한 머신러닝/딥러닝 프로젝트의 전체 과정을 경험하게 됩니다. 시험의 주요 목적은 리뷰 데이터를 기반으로 유용성 예측 문제를 정의하고, 텍스트 데이터를 활용해 다양한 모델을 구현하여 분석 및 비교하는 능력을 평가하는 것입니다. 실험 환경은 Python과 주요 라이브러리(Numpy, Pandas, Scikit-learn, TensorFlow/Keras 등)를 활용하여도 됩니다.

### **데이터 설명**

실험 환경을 고려하여 아마존 리뷰 데이터셋(https://amazon-reviews-2023.github.io/) 중 Health and personal care를 선택하시기 바랍니다. 선택한 데이터는 다음 두 가지 주요 컬럼으로 사용합니다.

1. 리뷰 본문(text)
    * 각 리뷰의 내용을 담고 있는 텍스트 데이터로, 예측 모델의 주요 특징(feature)으로 사용됩니다.

2. 유용성 투표 수(helpful_votes)
    * 리뷰가 유용한지 여부를 나타내는 숫자 값입니다.

## 문제 1: 데이터 탐색 및 전처리 (25점)

### 1.1 데이터 탐색 (15점)

리뷰 데이터를 탐색하고 다음 작업을 수행하시오.

In [ ]:
# 데이터셋 다운로드 (Health and Personal Care, 50k minified)
!wget https://datarepo.eng.ucsd.edu/mcauley_group/amazon_reviews_2023/json/minified/Health_and_Personal_Care_50000.json.gz

# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import gzip

# 데이터 로드 함수 정의
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

# 데이터프레임 생성
df = getDF('Health_and_Personal_Care_50000.json.gz')

# 'useful' 컬럼 생성: helpful_votes > 0 이면 1, 아니면 0
# NaN 값은 0으로 처리되도록 fillna(0)을 먼저 적용합니다.
df['useful'] = (df['helpful_votes'].fillna(0) > 0).astype(int)

# 'useful' 컬럼 분포 확인
print("'useful' 컬럼 분포:")
print(df['useful'].value_counts())

# 데이터프레임 상위 5개 행 출력
print("\n데이터프레임 상위 5개 행:")
print(df.head())

1. 리뷰의 유용성 투표 수(vote)를 기준으로 새로운 레이블을 생성하고, 레이블의 분포를 확인하시오.
    * helpful_votes > 0: 1 (유용한 리뷰)
    * helpful_votes = 0 또는 NaN: 0 (유용하지 않은 리뷰)

In [ ]:
# 리뷰 텍스트 길이 분석
import matplotlib.pyplot as plt

# 'text' 컬럼이 없는 경우를 대비하여 빈 문자열로 채움 (이전 데이터 로딩에서 text 컬럼은 기본적으로 존재함)
df['text'] = df['text'].fillna('')
df['review_length'] = df['text'].apply(len)

# 리뷰 텍스트 길이 분포 시각화
plt.figure(figsize=(10, 6))
plt.hist(df['review_length'], bins=50, color='skyblue', edgecolor='black')
plt.title('Distribution of Review Text Lengths')
plt.xlabel('Review Length (Number of Characters)')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.show()

# 주요 통계량 출력
print("\nKey Statistics for Review Text Length:")
print(f"Mean: {df['review_length'].mean():.2f}")
print(f"Median: {df['review_length'].median():.2f}")
print(f"Min: {df['review_length'].min()}")
print(f"Max: {df['review_length'].max()}")
print(df['review_length'].describe())

2. 리뷰 텍스트 길이의 분포를 확인하고, 평균, 최솟값, 최댓값 등 주요 통계량을 제시하시오.

In [ ]:
# 데이터 불균형 해결 및 학습/테스트 데이터 분할
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

# 데이터 불균형 문제 해결: 다운샘플링
# 현재 'useful' 레이블의 분포를 보면 0 (유용하지 않은 리뷰)이 1 (유용한 리뷰) 보다 훨씬 많습니다.
# 이러한 불균형은 모델이 다수 클래스에 편향되게 학습할 수 있습니다.
# 여기서는 다수 클래스인 '0'을 소수 클래스인 '1'의 수에 맞게 다운샘플링합니다.
# 다운샘플링은 정보 손실의 위험이 있지만, 계산 효율성이 좋고 구현이 간단합니다.
# 업샘플링은 과적합의 위험이 있을 수 있어, 우선 다운샘플링을 선택합니다.

df_majority = df[df['useful'] == 0]
df_minority = df[df['useful'] == 1]

print(f"\nOriginal majority class size: {len(df_majority)}")
print(f"Original minority class size: {len(df_minority)}")

# 다수 클래스를 소수 클래스의 크기로 다운샘플링
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # 비복원 추출
                                 n_samples=len(df_minority), # 소수 클래스 크기에 맞춤
                                 random_state=123) # 재현성을 위한 random_state

# 다운샘플링된 다수 클래스 데이터와 소수 클래스 데이터를 합침
df_balanced = pd.concat([df_majority_downsampled, df_minority])

print(f"\nBalanced dataset 'useful' column distribution:")
print(df_balanced['useful'].value_counts())

# 필터링된 데이터를 기준으로 학습 데이터와 테스트 데이터 분할
# 여기서 X는 리뷰 텍스트('text'), y는 'useful' 레이블입니다.
X = df_balanced['text'].fillna('') # Ensure no NaN values in text before splitting
y = df_balanced['useful']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123, stratify=y)
# stratify=y : 클래스 비율을 유지하면서 분할합니다.

print("\nShapes of the resulting datasets:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

3. 데이터 불균형 문제를 해결하기 위해 유용한 리뷰 수 기준으로 데이터를 샘플링 하시오.

4. 필터링된 데이터를 기준으로 학습 데이터와 테스트 데이터를 나누고 random_state=123으로 지정하시오.
※ 나눠진 학습 데이터와 테스트 데이터를 아래 실험에서 동일하게 사용하시오.

### 1.2 텍스트 전처리 (10점)

1. 리뷰 본문 데이터를 다음 과정을 포함하여 전처리하시오.
    * 소문자 변환, 특수문자 제거 등
    * 불용어 제거
    * 어간 추출(Stemming)과 표제어 추출(Lemmatization) 중 하나를 선택하여 적용하고 선택 근거를 서술하시오.

In [ ]:
# 텍스트 전처리
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# --- 선택 근거 --- 
# Lemmatization을 선택한 이유:
# Lemmatization은 단어의 기본형(lemma)을 찾아 변환합니다. 예를 들어 'running'을 'run'으로, 'better'를 'good'으로 바꿀 수 있습니다.
# 이는 Stemming(어간 추출)보다 정교한 방식으로, 단어의 의미를 보존하는 데 더 효과적입니다.
# Stemming은 종종 단어의 끝부분을 단순히 잘라내어 실제 단어가 아닌 형태(예: 'studies' -> 'studi')를 만들 수 있습니다.
# Lemmatization은 결과적으로 더 해석 가능하고 의미론적으로 정확한 토큰을 생성하여, 모델의 성능 향상에 기여할 수 있습니다.
# 계산 비용은 Stemming보다 다소 높지만, NLTK의 WordNetLemmatizer는 비교적 효율적이며, 전처리 단계에서의 약간의 추가 시간은 
# 모델 학습 및 예측의 정확도 향상으로 보상받을 수 있다고 판단됩니다.
# -------------------

def preprocess_text(text):
    # 1. 소문자 변환
    text = text.lower()
    # 2. 특수문자 제거 (알파벳과 공백만 남김)
    text = re.sub(r'[^a-z\s]', '', text)
    # 3. 토큰화 (공백 기준)
    words = text.split()
    # 4. 불용어 제거 및 표제어 추출(Lemmatization)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words and len(word) > 1] # 한 글자 단어도 제거
    return ' '.join(words)

# X_train 및 X_test의 각 리뷰에 전처리 함수 적용
# Series.copy()를 사용하여 SettingWithCopyWarning 방지
X_train_processed = X_train.copy().apply(preprocess_text)
X_test_processed = X_test.copy().apply(preprocess_text)

print("Text preprocessing completed.")
print(f"X_train_processed shape: {X_train_processed.shape}")
print(f"X_test_processed shape: {X_test_processed.shape}")

In [ ]:
# 전처리된 텍스트 데이터 확인 (예시 출력)
print("\n--- Preprocessing Examples (from X_train) ---")
for i in range(3):
    # X_train이 pandas Series이므로 .iloc를 사용하여 접근
    original_text = X_train.iloc[i]
    processed_text = X_train_processed.iloc[i]
    print(f"\nExample {i+1}:")
    print(f"Original: {original_text[:200]}...") # 처음 200자만 출력
    print(f"Processed: {processed_text[:200]}...") # 처음 200자만 출력

2. 전처리된 텍스트 데이터를 확인하고 몇 개의 예시를 포함하여 설명하시오.

## 문제 2: 머신러닝 기반 리뷰 유용성 예측 (30점)

### 2.1 TF-IDF 기반 특징 추출 (10점)

* 전처리된 리뷰 본문 데이터를 활용해 TF-IDF를 계산하시오.
* TF-IDF 상위 20개 단어를 시각화하거나 해석하고, 모델 학습에 적합한 데이터로 변환하시오.

In [ ]:
# TF-IDF 특징 추출
from sklearn.feature_extraction.text import TfidfVectorizer

# TfidfVectorizer 초기화
# max_features: TF-IDF 점수가 높은 상위 N개 단어만 사용 (여기서는 5000으로 제한)
# min_df: 단어가 나타나는 최소 문서 수 (너무 드문 단어 제외)
# max_df: 단어가 나타나는 최대 문서 비율 (너무 흔한 단어 제외, 예: 0.95는 95% 이상 문서에 나타나면 제외)
tfidf_vectorizer = TfidfVectorizer(max_features=5000, min_df=5, max_df=0.95)

# X_train_processed 데이터에 TfidfVectorizer를 학습(fit)시키고 변환(transform)
tfidf_train_features = tfidf_vectorizer.fit_transform(X_train_processed)

# X_test_processed 데이터에는 학습된 TfidfVectorizer를 사용하여 변환(transform)만 수행
tfidf_test_features = tfidf_vectorizer.transform(X_test_processed)

# 결과 TF-IDF 행렬의 형태 출력
print("Shape of TF-IDF matrix for training data:", tfidf_train_features.shape)
print("Shape of TF-IDF matrix for test data:", tfidf_test_features.shape)

In [ ]:
# 상위 TF-IDF 단어 확인
import numpy as np # numpy가 이미 import 되어있지만, 명시적으로 다시 import
import pandas as pd # pandas가 이미 import 되어있지만, 명시적으로 다시 import

# TfidfVectorizer로부터 단어 목록(feature names) 가져오기
feature_names = tfidf_vectorizer.get_feature_names_out()

# 훈련 데이터셋의 각 단어에 대한 TF-IDF 점수 합계 계산
# tfidf_train_features는 CSR(Compressed Sparse Row) 행렬이므로, toarray()로 변환 후 sum
sum_tfidf_scores = np.sum(tfidf_train_features.toarray(), axis=0)

# 단어와 해당 TF-IDF 점수 합계를 매핑하는 Series 생성
tfidf_scores_series = pd.Series(sum_tfidf_scores, index=feature_names)

# TF-IDF 점수 기준으로 내림차순 정렬하여 상위 20개 단어 추출
top_20_tfidf_words = tfidf_scores_series.sort_values(ascending=False).head(20)

print("\nTop 20 words by summed TF-IDF score across training documents:")
print(top_20_tfidf_words)

# 간단한 바 차트 시각화 (matplotlib 사용)
plt.figure(figsize=(12, 8))
top_20_tfidf_words.plot(kind='bar', color='teal')
plt.title('Top 20 Words by TF-IDF Score')
plt.xlabel('Words')
plt.ylabel('Summed TF-IDF Score')
plt.xticks(rotation=45, ha='right')
plt.tight_layout() # 레이아웃 조절
plt.show()

### 2.2 머신러닝 모델 구축 (15점)

* TF-IDF 데이터를 이용해 리뷰 유용성을 예측하는 분류 모델을 구현하시오.
* 최소 2가지 머신러닝 알고리즘(예: Logistic Regression, Random Forest)을 사용하고 성능을 Precision, Recall, F1-Score 기준으로 평가하시오.

In [ ]:
# Logistic Regression 모델 학습 및 평가
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support, classification_report

# Logistic Regression 모델 초기화 및 학습
lr_model = LogisticRegression(random_state=123, solver='liblinear', max_iter=1000) # solver와 max_iter 지정
lr_model.fit(tfidf_train_features, y_train)

# 테스트 데이터에 대한 예측
y_pred_lr = lr_model.predict(tfidf_test_features)

# 성능 평가 (Precision, Recall, F1-Score)
precision_lr, recall_lr, f1_score_lr, _ = precision_recall_fscore_support(y_test, y_pred_lr, average='macro')

print("--- Logistic Regression Model Performance ---")
print(f"Precision: {precision_lr:.4f}")
print(f"Recall: {recall_lr:.4f}")
print(f"F1-Score: {f1_score_lr:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))

In [ ]:
# Random Forest Classifier 모델 학습 및 평가
from sklearn.ensemble import RandomForestClassifier

# Random Forest 모델 초기화 및 학습
# n_estimators: 트리의 개수, class_weight='balanced'는 불균형 데이터에 도움될 수 있음 (여기서는 이미 다운샘플링함)
rf_model = RandomForestClassifier(n_estimators=100, random_state=123, class_weight='balanced_subsample', n_jobs=-1)
rf_model.fit(tfidf_train_features, y_train)

# 테스트 데이터에 대한 예측
y_pred_rf = rf_model.predict(tfidf_test_features)

# 성능 평가 (Precision, Recall, F1-Score)
precision_rf, recall_rf, f1_score_rf, _ = precision_recall_fscore_support(y_test, y_pred_rf, average='macro')

print("--- Random Forest Classifier Model Performance ---")
print(f"Precision: {precision_rf:.4f}")
print(f"Recall: {recall_rf:.4f}")
print(f"F1-Score: {f1_score_rf:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))

## 문제 3: CNN 기반 리뷰 유용성 예측 (40점)

### 3.1 임베딩 실험 (15점)

* 사전 학습된 워드 임베딩 모델(예: Word2Vec, GloVe)을 선택하고 이유를 서술하시오.
* 리뷰 길이에 따른 적절한 패딩 길이를 설정하고, 선택한 기준(예: 최대 길이, 평균 길이)을 설명하시오.
* 임베딩 차원을 [50, 100, 200]으로 변경하며 성능 변화를 실험하고, 최적의 임베딩 차원을 도출하시오.
* 사전 학습된 임베딩과 랜덤 초기화 임베딩(예: Keras 임베딩)을 비교하고 성능 차이를 분석하시오.

In [ ]:
# CNN용 텍스트 토큰화 및 패딩
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import numpy as np

# Tokenizer 초기화 (상위 10,000개 단어만 사용, OOV 토큰 사용)
vocab_size_cnn = 10000 
tokenizer = Tokenizer(num_words=vocab_size_cnn, oov_token='<oov>')

# X_train_processed 데이터에 Tokenizer를 학습
tokenizer.fit_on_texts(X_train_processed)

# 텍스트 데이터를 정수 시퀀스로 변환
X_train_sequences = tokenizer.texts_to_sequences(X_train_processed)
X_test_sequences = tokenizer.texts_to_sequences(X_test_processed)

# 시퀀스 길이 분석
sequence_lengths = [len(seq) for seq in X_train_sequences]
plt.figure(figsize=(10, 6))
plt.hist(sequence_lengths, bins=50, color='purple', edgecolor='black')
plt.title('Distribution of Sequence Lengths (X_train_processed)')
plt.xlabel('Sequence Length (Number of Tokens)')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.show()

percentile_95 = np.percentile(sequence_lengths, 95)
print(f"\n95th percentile of sequence lengths: {percentile_95:.2f}")

# maxlen 결정 근거:
# 대부분의 시퀀스 길이를 커버하면서도 너무 길지 않게 설정하기 위해 95번째 백분위수를 사용합니다.
# 이렇게 하면 대부분의 정보를 유지하면서 패딩으로 인한 계산 비효율성을 줄일 수 있습니다.
# 만약 95번째 백분위수가 너무 크거나 작다면, 평균 + 2*표준편차 등을 고려할 수 있으나, 여기서는 95th percentile을 기준으로 합니다.
maxlen = int(percentile_95) 
# maxlen = 200 # 또는 고정값으로 설정할 수 있습니다. 여기서는 95th percentile을 사용합니다.

# 시퀀스 패딩 (maxlen에 맞춰 길이를 통일)
X_train_padded = pad_sequences(X_train_sequences, maxlen=maxlen, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=maxlen, padding='post', truncating='post')

# 실제 사용된 단어 수 (Tokenizer가 num_words 제한을 적용하므로, word_index 크기와 다를 수 있음)
actual_vocab_size = min(vocab_size_cnn, len(tokenizer.word_index) + 1) # +1 for OOV token

print(f"\nVocabulary size (used by Tokenizer): {actual_vocab_size}")
print(f"Max sequence length (maxlen): {maxlen}")
print(f"Shape of X_train_padded: {X_train_padded.shape}")
print(f"Shape of X_test_padded: {X_test_padded.shape}")

In [ ]:
# 사전 학습된 GloVe 임베딩 로드 및 임베딩 행렬 생성 (100d 용으로 남겨둠, 다른 차원은 각 실험 셀에서 로드)
import os

# --- GloVe 임베딩 선택 근거 ---
# GloVe (Global Vectors for Word Representation)는 대규모 텍스트 코퍼스에서 단어 간의 동시 등장 통계를 기반으로 학습된 임베딩입니다.
# Word2Vec과 함께 널리 사용되는 임베딩 방식 중 하나로, 다양한 NLP 작업에서 좋은 성능을 보입니다.
# GloVe.6B.100d는 위키피디아와 Gigaword 5 코퍼스로 학습되었으며, 60억 토큰과 40만개의 단어 사전을 포함하고, 각 단어를 100차원 벡터로 표현합니다.
# 이는 일반적인 텍스트에 대한 의미 정보를 잘 포착하고 있어, 우리 데이터셋의 리뷰 텍스트에도 유용할 것으로 기대됩니다.
# FastText는 하위 단어(subword) 정보를 활용하여 OOV(Out-Of-Vocabulary) 단어에 강점이 있지만, GloVe도 충분히 넓은 어휘 범위를 가지고 있습니다.
# ---------------------------

# GloVe 파일 다운로드 (glove.6B.zip은 모든 차원을 포함하므로 한 번만 다운로드)
glove_zip_file = 'glove.6B.zip'
glove_url = 'http://nlp.stanford.edu/data/glove.6B.zip'
glove_files_to_check = ['glove.6B.50d.txt', 'glove.6B.100d.txt', 'glove.6B.200d.txt']

if not all(os.path.exists(f) for f in glove_files_to_check):
    if not os.path.exists(glove_zip_file):
        print(f"Downloading {glove_zip_file}...")
        !wget -q $glove_url
    print(f"Unzipping {glove_zip_file}...")
    !unzip -q -o $glove_zip_file # -o to overwrite if files exist from partial unzip
else:
    print(f"All required GloVe files already exist. Skipping download and unzip.")

# embedding_matrix와 embedding_dim은 각 실험 셀에서 설정됨
print("\nGloVe embedding base files should be available for subsequent cells.")

In [ ]:
# CNN 모델 정의 및 컴파일 (수정됨)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.metrics import Precision, Recall
import tensorflow as tf # F1Score를 위해 import

embedding_experiment_results = {} # 결과 저장용 딕셔너리

def create_cnn_model_flexible(vocab_size, embedding_dim, maxlen, embedding_matrix=None, trainable_embedding=False):
    input_layer = Input(shape=(maxlen,))
    
    if embedding_matrix is not None:
        # 사전 학습된 임베딩 사용
        embedding_layer = Embedding(input_dim=vocab_size, 
                                    output_dim=embedding_dim, 
                                    weights=[embedding_matrix], 
                                    input_length=maxlen, 
                                    trainable=trainable_embedding)(input_layer)
    else:
        # 랜덤 초기화 임베딩 사용
        embedding_layer = Embedding(input_dim=vocab_size, 
                                    output_dim=embedding_dim, 
                                    input_length=maxlen, 
                                    trainable=trainable_embedding)(input_layer)
    
    conv_layer = Conv1D(filters=128, kernel_size=5, activation='relu')(embedding_layer)
    pooling_layer = GlobalMaxPooling1D()(conv_layer)
    dropout_layer = Dropout(0.5)(pooling_layer) # Dropout 추가
    output_layer = Dense(1, activation='sigmoid')(dropout_layer)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy', 
                             Precision(name='precision'), 
                             Recall(name='recall'), 
                             tf.keras.metrics.F1Score(average='macro', name='f1_macro', threshold=0.5)])
    return model

print("Flexible CNN model creation function defined.")

# --- GloVe 100d Experiment (기존 모델 학습 및 평가) ---
print("\n--- Starting: CNN with GloVe 100d Embeddings ---")
glove_file_100d = 'glove.6B.100d.txt'
embedding_dim_100d = 100
embeddings_index_100d = {}
try:
    with open(glove_file_100d, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index_100d[word] = coefs
    print(f"Found {len(embeddings_index_100d)} word vectors in {glove_file_100d}.")

    embedding_matrix_100d = np.zeros((actual_vocab_size, embedding_dim_100d))
    hits_100d = 0
    misses_100d = 0
    for word, i in tokenizer.word_index.items():
        if i < actual_vocab_size:
            embedding_vector = embeddings_index_100d.get(word)
            if embedding_vector is not None:
                embedding_matrix_100d[i] = embedding_vector
                hits_100d += 1
            else:
                misses_100d += 1
    print(f"Embedding Matrix (100d) shape: {embedding_matrix_100d.shape}")
    print(f"Words converted (100d): {hits_100d}, Misses: {misses_100d}")

    cnn_model_glove_100d = create_cnn_model_flexible(actual_vocab_size, 
                                                 embedding_dim_100d, 
                                                 maxlen, 
                                                 embedding_matrix_100d, 
                                                 trainable_embedding=False)
    cnn_model_glove_100d.summary()
    print("\nTraining CNN with GloVe 100d...")
    history_100d = cnn_model_glove_100d.fit(X_train_padded, y_train, 
                                       epochs=10, 
                                       batch_size=64, 
                                       validation_data=(X_test_padded, y_test), 
                                       verbose=1)
    loss_100d, acc_100d, prec_100d, rec_100d, f1_100d = cnn_model_glove_100d.evaluate(X_test_padded, y_test, verbose=0)
    embedding_experiment_results['GloVe_100d'] = {'Loss': loss_100d, 'Accuracy': acc_100d, 'Precision': prec_100d, 'Recall': rec_100d, 'F1-Score': f1_100d}
    print(f"GloVe 100d - Test Loss: {loss_100d:.4f}, Accuracy: {acc_100d:.4f}, Precision: {prec_100d:.4f}, Recall: {rec_100d:.4f}, F1-Macro: {f1_100d:.4f}")

except FileNotFoundError:
    print(f"ERROR: {glove_file_100d} not found. Skipping 100d experiment.")
    embedding_experiment_results['GloVe_100d'] = {'Loss': np.nan, 'Accuracy': np.nan, 'Precision': np.nan, 'Recall': np.nan, 'F1-Score': np.nan}

In [ ]:
# --- GloVe 50d Experiment ---
print("\n--- Starting: CNN with GloVe 50d Embeddings ---")
glove_file_50d = 'glove.6B.50d.txt'
embedding_dim_50d = 50
embeddings_index_50d = {}
try:
    with open(glove_file_50d, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index_50d[word] = coefs
    print(f"Found {len(embeddings_index_50d)} word vectors in {glove_file_50d}.")

    embedding_matrix_50d = np.zeros((actual_vocab_size, embedding_dim_50d))
    hits_50d = 0
    misses_50d = 0
    for word, i in tokenizer.word_index.items():
        if i < actual_vocab_size:
            embedding_vector = embeddings_index_50d.get(word)
            if embedding_vector is not None:
                embedding_matrix_50d[i] = embedding_vector
                hits_50d += 1
            else:
                misses_50d += 1
    print(f"Embedding Matrix (50d) shape: {embedding_matrix_50d.shape}")
    print(f"Words converted (50d): {hits_50d}, Misses: {misses_50d}")

    cnn_model_glove_50d = create_cnn_model_flexible(actual_vocab_size, 
                                                embedding_dim_50d, 
                                                maxlen, 
                                                embedding_matrix_50d, 
                                                trainable_embedding=False)
    cnn_model_glove_50d.summary()
    print("\nTraining CNN with GloVe 50d...")
    history_50d = cnn_model_glove_50d.fit(X_train_padded, y_train, 
                                      epochs=10, 
                                      batch_size=64, 
                                      validation_data=(X_test_padded, y_test), 
                                      verbose=1)
    loss_50d, acc_50d, prec_50d, rec_50d, f1_50d = cnn_model_glove_50d.evaluate(X_test_padded, y_test, verbose=0)
    embedding_experiment_results['GloVe_50d'] = {'Loss': loss_50d, 'Accuracy': acc_50d, 'Precision': prec_50d, 'Recall': rec_50d, 'F1-Score': f1_50d}
    print(f"GloVe 50d - Test Loss: {loss_50d:.4f}, Accuracy: {acc_50d:.4f}, Precision: {prec_50d:.4f}, Recall: {rec_50d:.4f}, F1-Macro: {f1_50d:.4f}")

except FileNotFoundError:
    print(f"ERROR: {glove_file_50d} not found. Ensure 'glove.6B.zip' was downloaded and unzipped correctly. Skipping 50d experiment.")
    embedding_experiment_results['GloVe_50d'] = {'Loss': np.nan, 'Accuracy': np.nan, 'Precision': np.nan, 'Recall': np.nan, 'F1-Score': np.nan}

In [ ]:
# --- GloVe 200d Experiment ---
print("\n--- Starting: CNN with GloVe 200d Embeddings ---")
glove_file_200d = 'glove.6B.200d.txt'
embedding_dim_200d = 200
embeddings_index_200d = {}
try:
    with open(glove_file_200d, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index_200d[word] = coefs
    print(f"Found {len(embeddings_index_200d)} word vectors in {glove_file_200d}.")

    embedding_matrix_200d = np.zeros((actual_vocab_size, embedding_dim_200d))
    hits_200d = 0
    misses_200d = 0
    for word, i in tokenizer.word_index.items():
        if i < actual_vocab_size:
            embedding_vector = embeddings_index_200d.get(word)
            if embedding_vector is not None:
                embedding_matrix_200d[i] = embedding_vector
                hits_200d += 1
            else:
                misses_200d += 1
    print(f"Embedding Matrix (200d) shape: {embedding_matrix_200d.shape}")
    print(f"Words converted (200d): {hits_200d}, Misses: {misses_200d}")

    cnn_model_glove_200d = create_cnn_model_flexible(actual_vocab_size, 
                                                 embedding_dim_200d, 
                                                 maxlen, 
                                                 embedding_matrix_200d, 
                                                 trainable_embedding=False)
    cnn_model_glove_200d.summary()
    print("\nTraining CNN with GloVe 200d...")
    history_200d = cnn_model_glove_200d.fit(X_train_padded, y_train, 
                                       epochs=10, 
                                       batch_size=64, 
                                       validation_data=(X_test_padded, y_test), 
                                       verbose=1)
    loss_200d, acc_200d, prec_200d, rec_200d, f1_200d = cnn_model_glove_200d.evaluate(X_test_padded, y_test, verbose=0)
    embedding_experiment_results['GloVe_200d'] = {'Loss': loss_200d, 'Accuracy': acc_200d, 'Precision': prec_200d, 'Recall': rec_200d, 'F1-Score': f1_200d}
    print(f"GloVe 200d - Test Loss: {loss_200d:.4f}, Accuracy: {acc_200d:.4f}, Precision: {prec_200d:.4f}, Recall: {rec_200d:.4f}, F1-Macro: {f1_200d:.4f}")

except FileNotFoundError:
    print(f"ERROR: {glove_file_200d} not found. Ensure 'glove.6B.zip' was downloaded and unzipped correctly. Skipping 200d experiment.")
    embedding_experiment_results['GloVe_200d'] = {'Loss': np.nan, 'Accuracy': np.nan, 'Precision': np.nan, 'Recall': np.nan, 'F1-Score': np.nan}

In [ ]:
# --- Randomly Initialized Embedding Experiment (100d) ---
print("\n--- Starting: CNN with Randomly Initialized 100d Embeddings ---")
embedding_dim_random = 100 # Chosen dimension for random initialization

cnn_model_random_100d = create_cnn_model_flexible(actual_vocab_size, 
                                               embedding_dim_random, 
                                               maxlen, 
                                               embedding_matrix=None, # No pre-trained matrix
                                               trainable_embedding=True) # Embedding layer will be trained
cnn_model_random_100d.summary()
print("\nTraining CNN with Random 100d Embeddings...")
history_random_100d = cnn_model_random_100d.fit(X_train_padded, y_train, 
                                           epochs=10, 
                                           batch_size=64, 
                                           validation_data=(X_test_padded, y_test), 
                                           verbose=1)
loss_random, acc_random, prec_random, rec_random, f1_random = cnn_model_random_100d.evaluate(X_test_padded, y_test, verbose=0)
embedding_experiment_results['Random_100d'] = {'Loss': loss_random, 'Accuracy': acc_random, 'Precision': prec_random, 'Recall': rec_random, 'F1-Score': f1_random}
print(f"Random 100d - Test Loss: {loss_random:.4f}, Accuracy: {acc_random:.4f}, Precision: {prec_random:.4f}, Recall: {rec_random:.4f}, F1-Macro: {f1_random:.4f}")

### 임베딩 실험 결과 요약

다음은 다양한 임베딩 설정에 대한 CNN 모델의 성능을 요약한 표입니다.

| Embedding Type      | Dimension | Trainable | Test Loss | Test Accuracy | Test Precision | Test Recall | Test F1-Score (Macro) |
|---------------------|-----------|-----------|-----------|---------------|----------------|-------------|-----------------------|
| GloVe (Pre-trained) | 50        | False     | {:.4f}    | {:.4f}        | {:.4f}         | {:.4f}      | {:.4f}                |
| GloVe (Pre-trained) | 100       | False     | {:.4f}    | {:.4f}        | {:.4f}         | {:.4f}      | {:.4f}                |
| GloVe (Pre-trained) | 200       | False     | {:.4f}    | {:.4f}        | {:.4f}         | {:.4f}      | {:.4f}                |
| Random Initialized  | 100       | True      | {:.4f}    | {:.4f}        | {:.4f}         | {:.4f}      | {:.4f}                |

*표는 아래 셀에서 `embedding_experiment_results` 딕셔너리의 값으로 채워집니다.*

#### 결과 분석 및 최적 임베딩 차원 논의

**성능 비교:**

(이 부분은 실제 실험 결과가 나온 후 채워져야 합니다. 각 모델의 성능 지표를 비교하여 서술합니다.)

*   **GloVe 50d vs 100d vs 200d:** 일반적으로 임베딩 차원이 증가하면 더 많은 의미 정보를 표현할 수 있지만, 데이터셋 크기가 충분하지 않거나 작업의 복잡도에 비해 너무 큰 차원은 과적합을 유발하거나 성능 향상에 기여하지 못할 수 있습니다. 각 차원별 성능을 비교하여 어떤 차원이 가장 좋은 균형을 제공하는지 확인합니다.
*   **Pre-trained (GloVe 100d) vs Random Initialized (100d):** 사전 학습된 임베딩은 대규모 코퍼스에서 학습된 일반적인 언어 지식을 활용하므로, 특히 데이터셋이 작을 때 처음부터 학습하는 것보다 좋은 성능을 내는 경우가 많습니다. Randomly initialized embedding이 pre-trained embedding만큼의 성능을 내려면 더 많은 데이터와 학습 시간이 필요할 수 있습니다. 이 둘의 성능 차이를 통해 사전 학습된 임베딩의 효과를 평가합니다.

**최적 임베딩 설정:**

(이 부분도 실제 결과에 따라 달라집니다.)

예시:
*   만약 GloVe 100d가 가장 높은 F1-Score와 Accuracy를 보인다면, 이를 최적 설정으로 간주할 수 있습니다.
*   Randomly initialized embedding이 의외로 좋은 성능을 보인다면, 이는 데이터셋의 특정 어휘나 스타일에 잘 적응했기 때문일 수 있으며, `trainable=True`로 설정한 pre-trained embedding을 추가 실험해보는 것도 고려할 수 있습니다.
*   임베딩 차원이 높을수록 항상 성능이 좋아지는 것은 아니며, 특정 지점을 넘어서면 성능이 정체되거나 오히려 하락할 수 있습니다. 이 경우, 계산 효율성과 성능을 고려하여 적절한 차원을 선택합니다.

**결론:**

본 실험의 결과에 따르면, (가장 좋은 성능을 보인 설정)을 사용하는 것이 리뷰 유용성 예측 CNN 모델에 가장 적합한 임베딩 전략으로 보입니다. 그 이유는 (주요 성능 지표 및 비교 우위점 언급) 때문입니다.

In [ ]:
# 임베딩 실험 결과 표 채우기
g50 = embedding_experiment_results.get('GloVe_50d', {k: np.nan for k in ['Loss', 'Accuracy', 'Precision', 'Recall', 'F1-Score']})
g100 = embedding_experiment_results.get('GloVe_100d', {k: np.nan for k in ['Loss', 'Accuracy', 'Precision', 'Recall', 'F1-Score']})
g200 = embedding_experiment_results.get('GloVe_200d', {k: np.nan for k in ['Loss', 'Accuracy', 'Precision', 'Recall', 'F1-Score']})
r100 = embedding_experiment_results.get('Random_100d', {k: np.nan for k in ['Loss', 'Accuracy', 'Precision', 'Recall', 'F1-Score']})

summary_table_md = f"""
### 임베딩 실험 결과 요약

다음은 다양한 임베딩 설정에 대한 CNN 모델의 성능을 요약한 표입니다.

| Embedding Type      | Dimension | Trainable | Test Loss | Test Accuracy | Test Precision | Test Recall | Test F1-Score (Macro) |
|---------------------|-----------|-----------|-----------|---------------|----------------|-------------|-----------------------|
| GloVe (Pre-trained) | 50        | False     | {g50['Loss']:.4f}    | {g50['Accuracy']:.4f}        | {g50['Precision']:.4f}         | {g50['Recall']:.4f}      | {g50['F1-Score']:.4f}                |
| GloVe (Pre-trained) | 100       | False     | {g100['Loss']:.4f}    | {g100['Accuracy']:.4f}        | {g100['Precision']:.4f}         | {g100['Recall']:.4f}      | {g100['F1-Score']:.4f}                |
| GloVe (Pre-trained) | 200       | False     | {g200['Loss']:.4f}    | {g200['Accuracy']:.4f}        | {g200['Precision']:.4f}         | {g200['Recall']:.4f}      | {g200['F1-Score']:.4f}                |
| Random Initialized  | 100       | True      | {r100['Loss']:.4f}    | {r100['Accuracy']:.4f}        | {r100['Precision']:.4f}         | {r100['Recall']:.4f}      | {r100['F1-Score']:.4f}                |

#### 결과 분석 및 최적 임베딩 차원 논의

**성능 비교:**

(실제 실험 결과에 따라 이 부분을 상세히 채워주세요. 예를 들어, 각 모델의 F1-score, 정확도 등을 비교하고 어떤 모델이 특정 지표에서 우수한지 등을 설명합니다.)

*   **GloVe 50d vs 100d vs 200d:** GloVe 임베딩 차원을 50, 100, 200으로 변경하며 실험한 결과, (가장 좋은 성능을 보인 차원)d에서 (어떤 지표)가 가장 높게 나타났습니다. 예를 들어, 100d에서 F1-score가 가장 높았고, 200d에서는 오히려 약간 감소하거나 정체되는 경향을 보일 수 있습니다. 이는 데이터셋의 크기와 복잡도에 따라 적절한 임베딩 차원이 존재함을 시사합니다.
*   **Pre-trained (GloVe 100d) vs Random Initialized (100d):** 사전 학습된 GloVe 100d 임베딩을 사용한 모델과 랜덤 초기화된 100d 임베딩을 학습 과정에서부터 훈련시킨 모델을 비교했을 때, (GloVe 또는 Random 중 어떤 것이 더 나았는지) 모델이 (어떤 지표)에서 더 나은 성능을 보였습니다. 일반적으로 사전 학습된 임베딩이 더 좋은 초기값을 제공하여 빠르게 수렴하거나 더 나은 일반화 성능을 보이는 경향이 있지만, 데이터셋에 특화된 어휘가 많거나 데이터가 충분한 경우 랜덤 초기화 후 학습하는 것도 경쟁력 있는 결과를 낼 수 있습니다.

**최적 임베딩 설정:**

위 실험 결과를 종합적으로 고려했을 때, (가장 좋은 성능을 보인 설정, 예: GloVe 100d, trainable=False)이 현재 데이터셋과 모델 구조에서 가장 효과적인 임베딩 전략으로 판단됩니다. 해당 설정에서 F1-Score (Macro) {:.4f}, Accuracy {:.4f} 등의 성능을 기록했습니다. 
(만약 Randomly Initialized가 더 좋았다면 그 결과를 기술합니다.)

**결론:**

본 임베딩 실험을 통해, 사전 학습된 GloVe 임베딩을 사용하는 것이 랜덤 초기화 방식보다 (더 좋거나 비슷한, 또는 나쁜) 성능을 보였으며, 차원 중에서는 (50/100/200 중 최적 차원)d가 가장 균형 잡힌 결과를 제공했습니다. 따라서 향후 CNN 모델 실험에서는 (선택된 최적 임베딩 설정)을 기본으로 활용하는 것을 고려할 수 있습니다.
"""

# 이 마크다운 내용을 출력하거나, 혹은 노트북에 직접 마크다운 셀로 삽입하기 위해 준비합니다.
# 여기서는 print로 내용을 확인하고, 실제로는 이 내용을 바탕으로 마크다운 셀을 직접 수정하거나 추가합니다.
print("\n--- Embedding Experiment Summary Markdown Content ---")
print(summary_table_md.format(
    g50=g50, g100=g100, g200=g200, r100=r100, 
    best_setting_f1=embedding_experiment_results.get('GloVe_100d', {}).get('F1-Score', np.nan), # 예시로 GloVe 100d를 최적으로 가정
    best_setting_acc=embedding_experiment_results.get('GloVe_100d', {}).get('Accuracy', np.nan) # 예시로 GloVe 100d를 최적으로 가정
))

print("\nTo add this table to your notebook, copy the markdown content printed above ")
print("and paste it into a new Markdown cell. You will need to manually update the ")
print("discussion based on the actual numerical results obtained from the experiments.")

### 3.2 CNN 구조 실험 (15점)

#### CNN 구조 실험: 필터 크기 및 개수 변경

이전 임베딩 실험에서 (예시: GloVe 100d, non-trainable) 임베딩이 가장 좋은 성능을 보였다고 가정하고, 이를 고정하여 CNN 모델의 필터 크기와 필터 개수를 변경하며 실험을 진행합니다.
사용할 임베딩은 `embedding_matrix_100d` (100차원)이며, 임베딩 레이어는 학습되지 않도록(`trainable=False`) 설정합니다.

In [ ]:
# CNN 구조 실험: 필터 크기 및 개수 변경
from tensorflow.keras.layers import GlobalAveragePooling1D # For later use if needed

cnn_structure_experiment_results = []

def create_cnn_configurable_structure(vocab_size, embedding_dim, maxlen, embedding_matrix,
                                      num_filters_val, kernel_size_val, pooling_type='max',
                                      trainable_embedding=False):
    input_layer = Input(shape=(maxlen,))
    
    embedding_layer = Embedding(input_dim=vocab_size, 
                                output_dim=embedding_dim, 
                                weights=[embedding_matrix], 
                                input_length=maxlen, 
                                trainable=trainable_embedding)(input_layer)
    
    conv_layer = Conv1D(filters=num_filters_val, kernel_size=kernel_size_val, activation='relu')(embedding_layer)
    
    if pooling_type == 'max':
        pooling_layer = GlobalMaxPooling1D()(conv_layer)
    elif pooling_type == 'average':
        pooling_layer = GlobalAveragePooling1D()(conv_layer) # Average pooling
    else:
        raise ValueError("pooling_type must be 'max' or 'average'")
        
    dropout_layer = Dropout(0.5)(pooling_layer)
    output_layer = Dense(1, activation='sigmoid')(dropout_layer)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy', 
                             Precision(name='precision'), 
                             Recall(name='recall'), 
                             tf.keras.metrics.F1Score(average='macro', name='f1_macro', threshold=0.5)])
    return model

print("Configurable CNN model creation function defined for structure experiments.")

# --- CNN 구조 실험 루프 (필터 크기 및 개수) ---
# 가정: embedding_matrix_100d, embedding_dim_100d, actual_vocab_size, maxlen 사용
# 이 변수들은 이전 'cnn_model_definition_cell'에서 GloVe 100d에 대해 정의되어 있어야 합니다.
chosen_embedding_matrix = embedding_matrix_100d # 이전 단계에서 최적이라고 판단된 임베딩 행렬
chosen_embedding_dim = 100 # 해당 임베딩의 차원

kernel_sizes = [2, 3, 5]
num_filter_options = [32, 64, 128]

for ks in kernel_sizes:
    for nf in num_filter_options:
        print(f"\n--- Training CNN with Kernel Size: {ks}, Num Filters: {nf} (Max Pooling) ---")
        model_structure_cnn = create_cnn_configurable_structure(actual_vocab_size,
                                                              chosen_embedding_dim,
                                                              maxlen,
                                                              chosen_embedding_matrix,
                                                              num_filters_val=nf,
                                                              kernel_size_val=ks,
                                                              pooling_type='max',
                                                              trainable_embedding=False)
        #model_structure_cnn.summary() # 요약 출력을 반복해서 볼 필요는 없을 수 있음
        history = model_structure_cnn.fit(X_train_padded, y_train,
                                          epochs=10,
                                          batch_size=64,
                                          validation_data=(X_test_padded, y_test),
                                          verbose=0) # verbose=0으로 로그 줄임
        
        loss, acc, prec, rec, f1 = model_structure_cnn.evaluate(X_test_padded, y_test, verbose=0)
        print(f"Kernel: {ks}, Filters: {nf} (Max) -> Loss: {loss:.4f}, Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}")
        
        cnn_structure_experiment_results.append({
            'Kernel Size': ks,
            'Num Filters': nf,
            'Pooling Type': 'Max',
            'Loss': loss,
            'Accuracy': acc,
            'Precision': prec,
            'Recall': rec,
            'F1-Score': f1
        })

In [ ]:
# CNN 구조 실험 결과 DataFrame으로 표시
df_cnn_structure_results = pd.DataFrame(cnn_structure_experiment_results)
print("\n--- CNN Structure Experiment Results (Filter Size/Count) ---")
print(df_cnn_structure_results.sort_values(by='F1-Score', ascending=False))

#### CNN 구조 실험 결과 분석 (필터 크기 및 개수)

위 표는 GloVe 100차원 비학습 임베딩을 기반으로 필터 크기(Kernel Size)와 필터 개수(Num Filters)를 변경하며 실험한 CNN 모델의 성능을 보여줍니다.

**최적 조합:**

(실제 실행 후, 위 DataFrame 결과를 바탕으로 이 부분을 채워주세요. 예를 들어, F1-Score가 가장 높은 조합을 언급합니다.)

예시: `Kernel Size = 3`, `Num Filters = 128` 조합에서 F1-Score (Macro)가 0.7523으로 가장 높게 나타났습니다. 해당 조합의 다른 지표는 Accuracy 0.7600, Precision 0.7550, Recall 0.7580 입니다.

**결과 해석 및 근거:**

*   **필터 크기 (Kernel Size):**
    *   필터 크기는 한 번에 얼마나 많은 단어(토큰)를 컨볼루션 연산에 사용할지를 결정합니다. 작은 필터 크기(예: 2 또는 3)는 짧은 구문이나 바이그램/트라이그램(bigram/trigram) 패턴을 감지하는 데 유용합니다. 큰 필터 크기(예: 5)는 더 긴 구문 패턴을 포착할 수 있습니다.
    *   리뷰 데이터의 경우, 제품의 특징이나 감정을 나타내는 핵심 구문이 다양할 수 있습니다. (최적 필터 크기)가 좋은 성능을 보인 것은 해당 크기의 필터가 리뷰 텍스트에서 유용성 판단에 중요한 n-gram 패턴을 효과적으로 학습했기 때문일 수 있습니다. 예를 들어, 짧은 리뷰나 핵심 표현이 간결한 경우 작은 필터가, 문맥이 중요한 경우 중간 크기의 필터가 유리할 수 있습니다.

*   **필터 개수 (Num Filters):**
    *   필터 개수는 얼마나 다양한 특징을 학습할지를 결정합니다. 각 필터는 서로 다른 패턴을 감지하도록 학습됩니다. 필터 수가 많을수록 더 다양한 특징을 포착할 수 있지만, 모델의 복잡도가 증가하여 과적합의 위험이 있고 계산 비용도 커집니다.
    *   (최적 필터 개수)개의 필터를 사용했을 때 성능이 가장 좋았다면, 이는 해당 개수의 필터가 데이터의 복잡성을 충분히 표현하면서도 과적합을 피할 수 있는 적절한 수준이었음을 시사합니다. 너무 적은 필터는 중요한 특징을 놓칠 수 있고, 너무 많은 필터는 불필요한 노이즈까지 학습할 수 있습니다.

**결론:**

현재 데이터셋과 GloVe 100d (non-trainable) 임베딩을 사용한 CNN 모델에서는 필터 크기 (Kernel Size) (최적 값)과 필터 개수 (Num Filters) (최적 값)을 사용하는 것이 가장 좋은 성능을 보였습니다. 이는 해당 조합이 리뷰 텍스트의 유용성을 판단하는 데 필요한 특징들을 가장 효과적으로 추출하고 학습했음을 나타냅니다.

#### CNN 구조 실험: Pooling 전략 비교 (Max Pooling vs. Average Pooling)

앞선 필터 크기/개수 실험에서 도출된 최적의 조합 (예시: Kernel Size=3, Num Filters=128)과 GloVe 100d (non-trainable) 임베딩을 사용하여 Max Pooling과 Average Pooling 전략을 비교합니다.

In [ ]:
# CNN 구조 실험: Pooling 전략 비교
cnn_pooling_experiment_results = []

# 이전 실험에서 가장 좋았던 필터 크기와 개수를 사용합니다.
# df_cnn_structure_results가 존재하고 비어있지 않다고 가정합니다.
best_params_index = df_cnn_structure_results['F1-Score'].idxmax()
best_kernel_size = df_cnn_structure_results.loc[best_params_index, 'Kernel Size']
best_num_filters = df_cnn_structure_results.loc[best_params_index, 'Num Filters']

print(f"Using Best Kernel Size: {best_kernel_size}, Best Num Filters: {best_num_filters} for pooling experiments.")

pooling_strategies = ['max', 'average']

for pool_type in pooling_strategies:
    print(f"\n--- Training CNN with Pooling: {pool_type.upper()} ---")
    model_pooling_cnn = create_cnn_configurable_structure(actual_vocab_size,
                                                          chosen_embedding_dim, # GloVe 100d
                                                          maxlen,
                                                          chosen_embedding_matrix, # GloVe 100d matrix
                                                          num_filters_val=best_num_filters,
                                                          kernel_size_val=best_kernel_size,
                                                          pooling_type=pool_type,
                                                          trainable_embedding=False)
    # model_pooling_cnn.summary() # 요약은 필요시 활성화
    history = model_pooling_cnn.fit(X_train_padded, y_train,
                                      epochs=10,
                                      batch_size=64,
                                      validation_data=(X_test_padded, y_test),
                                      verbose=0)
    
    loss, acc, prec, rec, f1 = model_pooling_cnn.evaluate(X_test_padded, y_test, verbose=0)
    print(f"Pooling: {pool_type.upper()} -> Loss: {loss:.4f}, Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}")
    
    cnn_pooling_experiment_results.append({
        'Pooling Type': pool_type.upper(),
        'Loss': loss,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1
    })

In [ ]:
# CNN Pooling 전략 실험 결과 DataFrame으로 표시
df_cnn_pooling_results = pd.DataFrame(cnn_pooling_experiment_results)
print("\n--- CNN Pooling Strategy Experiment Results ---")
print(df_cnn_pooling_results.sort_values(by='F1-Score', ascending=False))

#### CNN 구조 실험 결과 분석 (Pooling 전략)

위 표는 최적의 필터 조합(Kernel Size: (값), Num Filters: (값))과 GloVe 100d (non-trainable) 임베딩을 사용하여 Max Pooling과 Average Pooling 전략을 비교한 결과입니다.

**성능 비교:**

(실제 실행 후, 위 DataFrame 결과를 바탕으로 이 부분을 채워주세요. 어떤 풀링 전략이 더 나은 성능을 보였는지 F1-Score와 Accuracy를 중심으로 비교합니다.)

예시:
*   Max Pooling 전략을 사용했을 때 F1-Score (Macro)는 0.7523, Accuracy는 0.7600을 기록했습니다.
*   Average Pooling 전략을 사용했을 때 F1-Score (Macro)는 0.7450, Accuracy는 0.7510을 기록했습니다.
*   이 경우, Max Pooling이 Average Pooling에 비해 (약간/상당히) 더 나은 성능을 보였습니다.

**각 전략의 장단점 및 선택 근거:**

*   **Max Pooling:**
    *   **장점:** 각 필터의 가장 강력한 신호(가장 큰 값)를 추출합니다. 이는 텍스트에서 가장 두드러지는 특징(예: 특정 키워드나 구문)을 감지하는 데 효과적일 수 있습니다. 위치에 민감하지 않은 특징을 잡아내는 데 유리하며, 노이즈가 있는 특징을 일부 무시하는 경향이 있습니다.
    *   **단점:** 필터가 감지한 다른 유용한 정보(덜 강하지만 여전히 의미 있는 신호들)를 잃을 수 있습니다.
    *   **선택 근거 (만약 Max Pooling이 우세했다면):** 리뷰 텍스트에서 유용성을 판단하는 데 있어 특정 핵심 구문이나 단어의 존재 여부가 전반적인 평균적인 신호보다 더 중요하게 작용했을 수 있습니다. Max Pooling은 이러한 결정적인 특징을 잘 포착하여 더 나은 분류 성능을 이끌어낸 것으로 보입니다.

*   **Average Pooling:**
    *   **장점:** 필터가 감지한 모든 신호의 평균을 사용하므로, 전반적인 특징을 부드럽게 요약합니다. 텍스트 전체에 걸쳐 분산된 정보를 종합하는 데 유리할 수 있습니다.
    *   **단점:** 중요한 특징이 약한 신호들에 의해 희석될 수 있습니다. Max Pooling만큼 특정 강한 신호를 명확하게 잡아내지 못할 수 있습니다.
    *   **선택 근거 (만약 Average Pooling이 우세했다면):** 리뷰의 전반적인 어조나 여러 부분에 걸쳐 나타나는 미묘한 신호들의 조합이 유용성 판단에 더 중요했을 수 있습니다. Average Pooling은 이러한 전체적인 정보를 더 잘 반영하여 성능 우위를 보였을 수 있습니다.

**결론:**

본 실험에서는 (Max/Average) Pooling 전략이 (F1-Score 등 주요 지표 언급)에서 더 나은 성능을 보였습니다. 이는 (선택된 풀링 전략)이 현재 데이터셋과 모델 구조에서 특징을 집약하는 데 더 효과적이었음을 시사합니다. 따라서 최종 CNN 모델 구성 시 (선택된 풀링 전략)을 사용하는 것이 바람직합니다.

* 필터 크기를 [2, 3, 5]로, 필터 개수를 [32, 64, 128]로 변경하며 성능 변화를 실험하시오.
* 리뷰 길이에 따른 최적 필터 크기를 도출하고 선택 근거를 서술하시오.
* Max Pooling과 Average Pooling 전략을 적용하여 모델 성능을 비교하고 각 전략의 장단점을 서술하시오.

### 3.3 추가 분석 (10점)

* 리뷰 길이에 따른 예측 성능 차이를 분석하시오(예: 짧은 리뷰 vs 긴 리뷰).

#### 리뷰 길이에 따른 성능 분석
이전 실험들에서 도출된 최적의 CNN 모델 설정을 사용하여 짧은 리뷰와 긴 리뷰에 대한 성능을 비교 분석합니다.
최적 설정 (가정): GloVe 100D (non-trainable), Kernel Size 3, Num Filters 128, Max Pooling. 
(실제 최적값은 이전 실험 결과에 따라 `df_cnn_structure_results` 및 `df_cnn_pooling_results`에서 가져와야 합니다.)

In [ ]:
# 최적 CNN 모델 파라미터 정의 및 모델 재학습
print("--- Re-training Best CNN Model for Short/Long Review Analysis ---")

# 이전 실험(df_cnn_structure_results, df_cnn_pooling_results)에서 최적값 로드 또는 가정
# 여기서는 예시값을 사용합니다. 실제로는 이전 DataFrame에서 idxmax() 등으로 찾아야 합니다.
best_kernel_size_final = df_cnn_structure_results.loc[df_cnn_structure_results['F1-Score'].idxmax(), 'Kernel Size'] if not df_cnn_structure_results.empty else 3
best_num_filters_final = df_cnn_structure_results.loc[df_cnn_structure_results['F1-Score'].idxmax(), 'Num Filters'] if not df_cnn_structure_results.empty else 128
# Pooling 결과에서 최적값 가져오기 (여기서는 Max가 더 좋았다고 가정)
best_pooling_type_final = df_cnn_pooling_results.loc[df_cnn_pooling_results['F1-Score'].idxmax(), 'Pooling Type'].lower() if not df_cnn_pooling_results.empty else 'max'

best_embedding_matrix = embedding_matrix_100d # GloVe 100d (cell cnn_model_definition_cell 에서 정의됨)
best_embedding_dim = 100 # GloVe 100d
is_embedding_trainable = False # GloVe 100d는 non-trainable로 가정

print(f"Using parameters: Kernel Size={best_kernel_size_final}, Num Filters={best_num_filters_final}, Pooling={best_pooling_type_final}")
print(f"Embedding: GloVe {best_embedding_dim}d, Trainable: {is_embedding_trainable}")

model_for_length_analysis = create_cnn_configurable_structure(
    actual_vocab_size, # defined in cnn_tokenization_padding_cell
    best_embedding_dim,
    maxlen, # defined in cnn_tokenization_padding_cell
    best_embedding_matrix,
    num_filters_val=best_num_filters_final,
    kernel_size_val=best_kernel_size_final,
    pooling_type=best_pooling_type_final,
    trainable_embedding=is_embedding_trainable
)

# model_for_length_analysis.summary() # 필요시 요약 출력

print("\nTraining the best model configuration for length analysis...")
history_best_model_for_length = model_for_length_analysis.fit(X_train_padded, y_train,
                                                              epochs=10,
                                                              batch_size=64,
                                                              validation_data=(X_test_padded, y_test),
                                                              verbose=0) # 로그 줄임

print("\nBest model for length analysis trained.")

In [ ]:
# 짧은 리뷰와 긴 리뷰 정의 및 데이터 분할
# X_test_sequences는 패딩 전 토큰화된 시퀀스 (cnn_tokenization_padding_cell에서 생성)
test_sequence_lengths = [len(seq) for seq in X_test_sequences]
median_length_test = np.median(test_sequence_lengths)

print(f"Median length of tokenized test sequences: {median_length_test}")

# 마스크 생성
short_reviews_mask = np.array([length <= median_length_test for length in test_sequence_lengths])
long_reviews_mask = np.array([length > median_length_test for length in test_sequence_lengths])

# X_test_padded와 y_test에 마스크 적용
X_test_padded_short = X_test_padded[short_reviews_mask]
y_test_short = y_test[short_reviews_mask]

X_test_padded_long = X_test_padded[long_reviews_mask]
y_test_long = y_test[long_reviews_mask]

print(f"Number of short reviews in test set: {len(X_test_padded_short)}")
print(f"Number of long reviews in test set: {len(X_test_padded_long)}")
print(f"Number of y_short_reviews_test: {len(y_test_short)}")
print(f"Number of y_long_reviews_test: {len(y_test_long)}")

In [ ]:
# 짧은 리뷰와 긴 리뷰에 대한 모델 성능 평가
print("\n--- Evaluating on Short Reviews ---")
results_short = {'Type': 'Short', 'Loss': np.nan, 'Accuracy': np.nan, 'Precision': np.nan, 'Recall': np.nan, 'F1-Score': np.nan}
if len(X_test_padded_short) > 0:
    loss_short, acc_short, prec_short, rec_short, f1_short = model_for_length_analysis.evaluate(X_test_padded_short, y_test_short, verbose=0)
    print(f"Short Reviews - Loss: {loss_short:.4f}, Accuracy: {acc_short:.4f}, Precision: {prec_short:.4f}, Recall: {rec_short:.4f}, F1-Macro: {f1_short:.4f}")
    results_short.update({'Loss': loss_short, 'Accuracy': acc_short, 'Precision': prec_short, 'Recall': rec_short, 'F1-Score': f1_short})
else:
    print("No short reviews to evaluate.")

print("\n--- Evaluating on Long Reviews ---")
results_long = {'Type': 'Long', 'Loss': np.nan, 'Accuracy': np.nan, 'Precision': np.nan, 'Recall': np.nan, 'F1-Score': np.nan}
if len(X_test_padded_long) > 0:
    loss_long, acc_long, prec_long, rec_long, f1_long = model_for_length_analysis.evaluate(X_test_padded_long, y_test_long, verbose=0)
    print(f"Long Reviews - Loss: {loss_long:.4f}, Accuracy: {acc_long:.4f}, Precision: {prec_long:.4f}, Recall: {rec_long:.4f}, F1-Macro: {f1_long:.4f}")
    results_long.update({'Loss': loss_long, 'Accuracy': acc_long, 'Precision': prec_long, 'Recall': rec_long, 'F1-Score': f1_long})
else:
    print("No long reviews to evaluate.")

length_analysis_results_df = pd.DataFrame([results_short, results_long])
print("\n--- Performance by Review Length ---")
print(length_analysis_results_df.to_string())

#### 리뷰 길이에 따른 성능 분석 결과 논의

다음 표는 최적의 CNN 모델을 사용하여 짧은 리뷰와 긴 리뷰 (테스트셋의 토큰화된 시퀀스 길이 중앙값 기준)에 대한 성능을 비교한 결과입니다.

```
{length_analysis_table}
```

*위 표는 이전 코드 셀의 `length_analysis_results_df` DataFrame 값으로 채워집니다.*

**관찰된 차이점:**

(실제 실험 결과에 따라 이 부분을 상세히 기술합니다. 예를 들어, 모델이 긴 리뷰에서 더 나은 성능을 보였는지, 아니면 짧은 리뷰에서 더 나았는지, 혹은 큰 차이가 없는지 등을 설명합니다.)

*   **정확도 (Accuracy):** (짧은 리뷰 vs 긴 리뷰 정확도 비교 및 설명)
*   **F1-Score (Macro):** (짧은 리뷰 vs 긴 리뷰 F1-score 비교 및 설명)
*   **정밀도 (Precision) 및 재현율 (Recall):** 각 클래스(0과 1) 또는 매크로 평균에 대해 정밀도와 재현율이 리뷰 길이에 따라 어떻게 변하는지 관찰합니다. 예를 들어, 긴 리뷰에서 특정 클래스를 더 잘 예측하거나, 짧은 리뷰에서 다른 클래스를 더 잘 예측할 수 있습니다.

**잠재적인 이유:**

*   **긴 리뷰의 경우:**
    *   **장점:** 더 많은 컨텍스트와 정보를 포함하고 있어 모델이 유용성을 판단하는 데 필요한 단서를 더 많이 찾을 수 있습니다. 다양한 n-gram 패턴이 나타날 가능성이 높습니다.
    *   **단점:** 핵심 내용과 관련 없는 노이즈가 많거나, 너무 길어서 `maxlen` 설정으로 인해 중요한 정보가 잘릴 수 있습니다. 또한, 문맥이 복잡해져 모델이 혼란스러워할 수도 있습니다.
*   **짧은 리뷰의 경우:**
    *   **장점:** 핵심 내용이 간결하게 담겨 있을 가능성이 높습니다. 노이즈가 적을 수 있습니다.
    *   **단점:** 컨텍스트가 부족하여 모델이 유용성을 판단하기 어려울 수 있습니다. 특히, 미묘한 감정이나 간접적인 표현을 이해하기 어려울 수 있습니다.

**모델 아키텍처와의 관계:**

CNN 모델은 지역적 특징(local features)을 감지하는 데 능합니다. 사용된 필터 크기 (예: 3 또는 5)는 특정 길이의 n-gram 패턴을 포착합니다. 
*   만약 모델이 긴 리뷰에서 더 좋은 성능을 보인다면, 이는 긴 리뷰에 중요한 패턴이 더 많이 포함되어 있고, `GlobalMaxPooling1D`가 이러한 다양한 패턴 중에서 가장 강력한 신호를 잘 선택했기 때문일 수 있습니다.
*   만약 짧은 리뷰에서 더 좋은 성능을 보인다면, 이는 짧은 리뷰의 간결함이 모델이 핵심 특징에 집중하는 데 도움이 되었거나, 긴 리뷰의 복잡성/노이즈가 모델을 방해했을 수 있습니다. `maxlen`으로 인한 정보 손실도 짧은 리뷰에 비해 긴 리뷰에서 더 클 수 있습니다.

**결론:**

종합적으로, 이 모델은 (짧은/긴/비슷한) 리뷰에서 더 나은 성능을 보이는 경향이 있습니다. 이는 (위에서 논의된 이유 중 가장 설득력 있는 내용 요약) 때문으로 해석할 수 있습니다. 리뷰 길이에 따른 이러한 성능 차이는 모델 개선 시 고려해야 할 중요한 요소입니다 (예: 리뷰 길이에 따라 다른 처리 방식을 적용하거나, 더 긴 시퀀스를 처리할 수 있는 아키텍처 고려 등).

In [ ]:
# 리뷰 길이에 따른 성능 분석 결과 표 채우기 위한 코드
if 'length_analysis_results_df' in globals() and not length_analysis_results_df.empty:
    table_string = length_analysis_results_df.to_markdown(index=False)
    print("\n--- Markdown Table for Length Analysis Results ---")
    print(table_string)
    print("\nCopy the table above and paste it into the markdown cell 'short_long_discussion_cell' replacing the placeholder line.")
else:
    print("\n`length_analysis_results_df` not found or is empty. Cannot generate markdown table.")

## 문제 4: 모델 비교 및 결론 (20점)

### 4.1 모델 비교 분석 (10점)

* TF-IDF 기반 모델과 CNN 기반 모델의 성능을 비교하시오.

### 4.2 결론 및 제언 (10점)

* 리뷰 텍스트 특성(리뷰 길이)과 유용성의 관계에 대한 통찰을 서술하시오.
* 모델 성능 향상을 위한 추가 개선 방향을 제안하시오.